In [213]:
# import packages
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
from scipy import stats
import statistics
import plotly.graph_objects as go
from scipy.stats import norm
from plotly.subplots import make_subplots
from sklearn.neighbors import KernelDensity

In [214]:
# Load the market prices
S_P100index = pd.read_excel("/files/exercises/Homeworks/HW4/TP4.xls", sheet_name = 'S&P100Index', skiprows = 4)
S_P100index.columns = ['date','S&P100','S&P500']
S_P100index = S_P100index.set_index('date')

# Load the stocks prices
S_P100const = pd.read_excel("/files/exercises/Homeworks/HW4/TP4.xls", sheet_name = 'S&P100Constituents', skiprows = 4)
S_P100const.rename(columns = {'Code': 'date'}, inplace=True)
S_P100const = S_P100const.set_index('date')

# Load the risk free rates
TBill3Months = pd.read_excel("/files/exercises/Homeworks/HW4/TP4.xls", sheet_name = 'TBill3Months', skiprows = 4)
TBill3Months.columns = ['date','US bill 3m']
TBill3Months = TBill3Months.set_index('date')

# Load something
FamaFrenchPortfolios = pd.read_excel("/files/exercises/Homeworks/HW4/TP4.xls", sheet_name = 'FamaFrenchPortfolios', skiprows = 21)
FamaFrenchPortfolios.columns = ['date','Small_Low BE/ME', 'Small_Med BE/ME', 'Small_High BE/ME','Big_Low BE/ME', 'Big_Med BE/ME', 'Big_High BE/ME']
FamaFrenchPortfolios = FamaFrenchPortfolios.set_index('date')


In [215]:
# Compute Arithmetic return for each market
AR_100 = pd.DataFrame((S_P100index.iloc[1:,0:].values - S_P100index.iloc[0:-1,0:].values) / S_P100index.iloc[0:-1,0:].values,  columns = ['S&P100','S&P500'])

# Compute Arithmetic return for each stocks
AR_100_const = pd.DataFrame((S_P100const.iloc[1:,0:].values - S_P100const.iloc[0:-1,0:].values) / S_P100const.iloc[0:-1,0:].values)

# Compute risk free rates (since it's given annualy we have to divide it by 100 and 52 to get weekly's rates)
AR_US3M = pd.DataFrame((TBill3Months.iloc[1:,0:].values / (100*52)),  columns = ['US 3 month'])

# Test of the CAPM : Time Series approach

## Exercice 1

In [217]:
# Compute the index of the return of interest
index_start = int(np.where(S_P100index.index == '1992-11-12')[0])
index_end = int(np.where(S_P100index.index == '2001-08-16')[0] - 1)

In [218]:
# Compute the beta of each stock thanks to the formula cov(ri,rm)/var(rm)
beta_stock = np.ones(AR_100_const.shape[1])
for i in range(0, AR_100_const.shape[1]):
    beta_stock[i] = np.cov(AR_100_const.iloc[index_start:index_end, i].values, AR_100.iloc[index_start:index_end, 1].values)[0, 1] / np.cov(AR_100_const.iloc[index_start:index_end, i].values, AR_100.iloc[index_start:index_end, 1].values)[1, 1]
beta_stock

array([1.01282832, 0.61692253, 0.70226629, 0.26288461, 1.5011201 ,
       1.19411279, 1.00994052, 0.42818723, 1.88042108, 0.90722787,
       0.65578464, 0.62306518, 1.21958741, 1.21072348, 0.69737981,
       0.98908595, 0.6996654 , 0.91723577, 0.63606875, 0.66963788,
       0.46620781, 0.75001695, 1.72275657, 1.8230663 , 1.1711563 ,
       0.74115045, 0.85334759, 1.13587674, 1.01787899, 0.60729716,
       0.77763267, 0.59554323, 0.50778921, 1.79608723, 0.1663694 ,
       0.19964571, 0.41426921, 1.1507459 , 0.82336302, 0.51092668,
       1.25650684, 0.90455161, 0.66070636, 0.84820587, 1.04090396,
       0.80745166, 0.37216813, 1.34582361, 1.39519633, 1.09302929,
       1.30353747, 0.99158549, 0.63436943, 1.52609012, 0.67611415,
       0.86039635, 0.91612017, 0.68343044, 1.16773832, 1.11395686,
       0.72394591, 1.88697348, 1.21178647, 1.32617352, 1.97903644,
       0.71619639, 1.57308012, 0.56170473, 0.88381527, 0.74861533,
       0.50041142, 0.49944865, 1.408097  , 0.5003508 , 0.85799

## Exercice 2

In [207]:
# Compute the alphas and betas of each stock thanks to the linear regression
nb_asset = len(AR_100_const.columns) # Compute the number of colums to deal with
betas = np.zeros(nb_asset) # Initialize an array with nb_asset entries to receive the beta
alphas = np.zeros(nb_asset) # Initialize an array with nb_asset entries to receive the alpha
std_alpha = np.zeros(nb_asset) # Initialize an array with nb_asset entries to receive the std of alpha

for i in range(0, nb_asset):
    zi =  AR_100_const.iloc[index_start:index_end, i].values - AR_US3M.iloc[index_start:index_end, 0].values
    zm = AR_100.iloc[index_start:index_end, 1].values - AR_US3M.iloc[index_start:index_end, 0].values
    X = sm.add_constant(zm)
    y = zi
    
    # Fit regression on the data of interest and save the alpha, beta and std of alpha
    reg = sm.OLS(endog = y, exog = X, missing='drop')
    results = reg.fit()
    betas[i] = results.params[1]
    alphas[i] = results.params[0]
    std_alpha[i] = results.bse[0]

In [208]:
# Compute the t-value of the alpha (if absolut value < 1.96 --> can not reject H0 (alpha = 0))
np.around(abs(alphas/std_alpha), decimals=2)

array([1.68, 1.13, 0.91, 0.05, 1.06, 1.52, 1.34, 0.94, 3.29, 0.38, 0.78,
       0.29, 0.14, 0.66, 0.89, 0.17, 0.77, 0.05, 0.94, 0.18, 0.3 , 1.29,
       1.95, 1.94, 2.88, 0.23, 0.82, 0.54, 0.36, 0.21, 0.06, 0.14, 0.89,
       2.14, 0.12, 0.62, 0.86, 0.55, 0.76, 0.98, 1.73, 0.35, 0.06, 0.42,
       0.58, 0.92, 0.07, 0.8 , 0.87, 0.3 , 1.86, 1.4 , 0.26, 0.34, 1.23,
       0.02, 0.36, 0.39, 1.57, 1.4 , 0.65, 1.07, 1.85, 0.73, 0.32, 0.67,
       2.13, 0.52, 1.51, 1.04, 0.25, 0.67, 0.69, 0.13, 0.36, 0.22, 0.54,
       0.13, 0.39, 0.72, 1.48, 0.96, 2.09, 1.45, 0.2 , 1.58, 0.32, 0.31,
       0.63, 0.02, 0.88, 0.13, 1.29, 0.53, 0.29])

In [209]:
# Compute the number of stock (over 95 = nb_asset) with alpha = 0 (statistically speaking)
np.sum(abs(alphas/std_alpha) < 1.96)

90

In [210]:
# Fit regression on the remaining data and save the r-squared of each stocks
index_start2 = int(np.where(S_P100index.index == '2001-08-16')[0] - 1)
r2 = np.zeros(nb_asset)

for i in range(0, nb_asset):
    zi =  AR_100_const.iloc[index_start2:, i].values - AR_US3M.iloc[index_start2:, 0].values
    zm = AR_100.iloc[index_start2:, 1].values - AR_US3M.iloc[index_start2:, 0].values
    X = sm.add_constant(zm)
    y = zi
    
    reg = sm.OLS(endog = y, exog = X, missing='drop')
    results = reg.fit()
    r2[i] = results.rsquared

In [211]:
# Compute the r-squared for each stocks
np.around(r2, decimals=2)

array([0.19, 0.64, 0.39, 0.05, 0.73, 0.57, 0.09, 0.06, 0.48, 0.32, 0.04,
       0.29, 0.53, 0.67, 0.02, 0.31, 0.22, 0.55, 0.35, 0.46, 0.07, 0.12,
       0.36, 0.75, 0.42, 0.01, 0.03, 0.3 , 0.56, 0.31, 0.41, 0.35, 0.32,
       0.3 , 0.1 , 0.07, 0.49, 0.36, 0.43, 0.02, 0.59, 0.55, 0.09, 0.22,
       0.34, 0.02, 0.04, 0.39, 0.42, 0.44, 0.4 , 0.42, 0.55, 0.62, 0.25,
       0.35, 0.42, 0.36, 0.36, 0.28, 0.36, 0.71, 0.58, 0.51, 0.07, 0.43,
       0.28, 0.12, 0.35, 0.14, 0.09, 0.02, 0.28, 0.  , 0.49, 0.16, 0.21,
       0.32, 0.1 , 0.  , 0.51, 0.45, 0.31, 0.45, 0.52, 0.45, 0.22, 0.57,
       0.36, 0.43, 0.39, 0.53, 0.15, 0.29, 0.45])

In [212]:
# Compute the average r-squared of each stocks
statistics.mean(np.around(r2, decimals=2))

0.32789473684210524